In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from string import digits
import os 
import cv2
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
import random
import datetime
from tensorflow import keras


In [2]:
os.listdir('data/')

['Mouth2901.jpg',
 'Ear3358.jpg',
 'Ear3359.jpg',
 'Ear3360.jpg',
 'Ear3361.jpg',
 'Ear3362.jpg',
 'Ear3363.jpg',
 'Ear3364.jpg',
 'Ear3365.jpg',
 'Ear3366.jpg',
 'Ear3367.jpg',
 'Ear3368.jpg',
 'Ear1.jpg',
 'Ear2.jpg',
 'Ear3.jpg',
 'Ear4.jpg',
 'Ear5.jpg',
 'Ear6.jpg',
 'Ear7.jpg',
 'Ear8.jpg',
 'Ear9.jpg',
 'Ear10.jpg',
 'Ear11.jpg',
 'Ear12.jpg',
 'Ear13.jpg',
 'Ear14.jpg',
 'Ear15.jpg',
 'Ear16.jpg',
 'Ear17.jpg',
 'Ear18.jpg',
 'Ear19.jpg',
 'Ear20.jpg',
 'Ear21.jpg',
 'Ear22.jpg',
 'Ear23.jpg',
 'Ear24.jpg',
 'Ear25.jpg',
 'Ear26.jpg',
 'Ear27.jpg',
 'Ear28.jpg',
 'Ear29.jpg',
 'Ear30.jpg',
 'Ear31.jpg',
 'Ear32.jpg',
 'Ear33.jpg',
 'Ear34.jpg',
 'Ear35.jpg',
 'Ear36.jpg',
 'Ear37.jpg',
 'Ear38.jpg',
 'Ear39.jpg',
 'Ear40.jpg',
 'Ear41.jpg',
 'Ear42.jpg',
 'Ear43.jpg',
 'Ear44.jpg',
 'Ear45.jpg',
 'Ear46.jpg',
 'Ear47.jpg',
 'Ear48.jpg',
 'Ear49.jpg',
 'Ear50.jpg',
 'Ear51.jpg',
 'Ear52.jpg',
 'Ear53.jpg',
 'Ear54.jpg',
 'Ear55.jpg',
 'Ear56.jpg',
 'Ear57.jpg',
 'Ear58.jpg',
 'E

In [3]:
label_dict = {'Ear':0, 'Eye':1, 'Hand':2, 'Mouth':3, 'Nose':4}

inverse_dict = {v: k for k, v in label_dict.items()}

remove_digits = str.maketrans('', '', digits)

In [4]:
example_image = 'data\\Mouth1.jpg'

example_image.split('.')[0].split('\\')[1].translate(remove_digits)

'Mouth'

In [5]:
def preprocess_images(img):

    image = cv2.imread(img)
    resized = cv2.resize(image, (128,128))

    label_temp = img.split('.')[0].split('\\')[1].translate(remove_digits)
    label = label_dict[label_temp]
    return resized/255, label

In [6]:
preprocess_images('data\\Mouth255.jpg')[0]

array([[[0.72156863, 0.75686275, 0.80784314],
        [0.71764706, 0.75294118, 0.80392157],
        [0.71764706, 0.75686275, 0.79607843],
        ...,
        [0.21176471, 0.12156863, 0.11372549],
        [0.21960784, 0.12941176, 0.12156863],
        [0.21960784, 0.12941176, 0.12156863]],

       [[0.71764706, 0.75294118, 0.80392157],
        [0.71764706, 0.75294118, 0.80392157],
        [0.71764706, 0.75686275, 0.79607843],
        ...,
        [0.21176471, 0.12156863, 0.11372549],
        [0.21568627, 0.1254902 , 0.11764706],
        [0.21568627, 0.1254902 , 0.11764706]],

       [[0.71764706, 0.75686275, 0.79607843],
        [0.71372549, 0.75294118, 0.79215686],
        [0.70980392, 0.74901961, 0.78823529],
        ...,
        [0.21176471, 0.12156863, 0.11372549],
        [0.21568627, 0.1254902 , 0.11764706],
        [0.21176471, 0.12156863, 0.11372549]],

       ...,

       [[0.64705882, 0.56470588, 0.55686275],
        [0.65882353, 0.57647059, 0.56862745],
        [0.65098039, 0

In [7]:
def load_images():
    images, labels = [], []

    for i in os.listdir('data'):
        processed_img, label_img = preprocess_images(os.path.join('data', i))


        images.append(processed_img)
        labels.append(label_img)
    return np.array(images), np.array(labels)    

In [8]:
image_data, label_data = load_images()

In [36]:
image_data.shape

(16561, 128, 128, 3)

In [9]:
random.random()

0.5376983433948119

In [10]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [11]:
def create_partition(img, lab):
    x_train, y_train, x_val, y_val = [], [], [], [],

    for i in zip(img, lab):
        rand = random.random()
        if rand > 0.1:
            x_train.append(i[0])
            y_train.append(i[1])
        else:
            x_val.append(i[0])
            y_val.append(i[1])
        
    assert len(x_train) == len(y_train)

    return np.array(x_train), np.array(y_train), np.array(x_val), np.array(y_val)             

In [12]:
x_train, y_train, x_val, y_val = create_partition(image_data, label_data)

In [13]:
x_train.shape

(14919, 128, 128, 3)

In [14]:
def build_model():

    model = tf.keras.models.Sequential()

    model.add(tf.keras.layers.Conv2D(16, (3,3),padding='same', activation='relu', input_shape=(128,128,3)))
    model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=1))
    model.add(tf.keras.layers.Conv2D(32, (3,3),padding='same', activation='relu'))
    model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
    model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dense(64, activation='relu'))
    model.add(tf.keras.layers.Dense(5, activation='softmax'))
    
    return model

In [15]:
model = build_model()
name = datetime.datetime.now().strftime('%H-%M-%S')

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 127, 127, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 63, 63, 32)       0         
 2D)                                                             
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 31, 31, 32)       0         
 2D)                                                             
                                                        

In [17]:
stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                        patience = 5,
                                        mode = 'min',
                                        restore_best_weights=True)

tensorboard = tf.keras.callbacks.TensorBoard(log_dir=f'log/{name}')                                        

In [18]:
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=1e-3),
              loss='sparse_categorical_crossentropy',
              metrics= ['accuracy'])

In [19]:
x_train = x_train.reshape(x_train.shape[0], 128,128,3)
x_val = x_val.reshape(x_val.shape[0], 128,128,3)
y_train = y_train.reshape(-1,1)
y_val = y_val.reshape(-1,1)

In [20]:
history = model.fit(x=x_train, y=y_train,batch_size=128, validation_data=(x_val,y_val), epochs=10, callbacks=[stop, tensorboard])

Epoch 1/10
117/117 [==============================] - 178s 1s/step - loss: 1.4179 - accuracy: 0.5271 - val_loss: 1.1694 - val_accuracy: 0.6060
Epoch 2/10
117/117 [==============================] - 167s 1s/step - loss: 0.8525 - accuracy: 0.7574 - val_loss: 0.6639 - val_accuracy: 0.7966
Epoch 3/10
117/117 [==============================] - 167s 1s/step - loss: 0.5650 - accuracy: 0.8311 - val_loss: 0.4964 - val_accuracy: 0.8642
Epoch 4/10
117/117 [==============================] - 159s 1s/step - loss: 0.4124 - accuracy: 0.8778 - val_loss: 0.3935 - val_accuracy: 0.8837
Epoch 5/10
117/117 [==============================] - 151s 1s/step - loss: 0.3053 - accuracy: 0.9151 - val_loss: 0.3386 - val_accuracy: 0.8855
Epoch 6/10
117/117 [==============================] - 166s 1s/step - loss: 0.2407 - accuracy: 0.9373 - val_loss: 0.2089 - val_accuracy: 0.9562
Epoch 7/10
117/117 [==============================] - 165s 1s/step - loss: 0.2000 - accuracy: 0.9515 - val_loss: 0.1712 - val_accuracy: 0.9695

In [18]:
model.predict_generator

<bound method Model.predict_generator of <keras.engine.sequential.Sequential object at 0x000001C303F7AB20>>

In [25]:
model_json=model.to_json()
with open("model.json",'w') as file:
    file.write(model_json)
model.save_weights("model.h5")    

In [2]:
from keras.models import model_from_json
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()

load_model = model_from_json(loaded_model_json)
load_model.load_weights("model.h5")

In [6]:
def predict_image(img):
  img_4d=img.reshape(1,128,128,3)
  prediction=model.predict(img_4d)[0]
  return {inverse_dict[i]: float(prediction[i]) for i in range(5)}

In [7]:
import gradio as gr

In [ ]:
image = gr.inputs.Image(shape=(128,128))
label = gr.outputs.Label(num_top_classes=5)

gr.Interface(fn=predict_image, inputs=image, outputs=label,interpretation='default').launch(debug='True')